## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/tips.csv"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.csv(file_location,header=True,inferSchema=True)

In [0]:
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [0]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [0]:
from pyspark.ml.feature import StringIndexer
indexer=StringIndexer(inputCols=['sex','smoker','day','time'],outputCols=["{}_indexed".format(c) for c in ['sex','smoker','day','time']])
df_r=indexer.fit(df).transform(df)
df_r.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|


In [0]:
df_r.columns



Out[12]: ['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_indexed',
 'smoker_indexed',
 'day_indexed',
 'time_indexed']

In [0]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(
inputCols=['tip','size','sex_indexed','smoker_indexed','day_indexed','time_indexed'],
outputCol='total_bill_new'
)

output=featureassembler.transform(df_r)
output.show()



+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|      total_bill_new|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|[1.01,2.0,1.0,0.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[1.66,3.0,0.0,0.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[3.5,3.0,0.0,0.0,...|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|[3.31,2.0,0.0,0.0...|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|[3.61,4.0,1.0,0.0...|
|     25.29|4.71|  Male|    No|S

In [0]:
finalized_data=output.select('total_bill_new','total_bill')

In [0]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(
featuresCol='total_bill_new',
labelCol='total_bill'
)
regressor=regressor.fit(train_data)

In [0]:
pred_result=regressor.evaluate(test_data)
pred_result.predictions.show()

+--------------------+----------+------------------+
|      total_bill_new|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,1],[1.75,2.0])|     17.82| 13.59635721826937|
| (6,[0,1],[2.0,2.0])|     12.69|14.423917712220646|
|(6,[0,1],[2.24,3.0])|     16.04|18.525540005295916|
|(6,[0,1],[2.72,2.0])|     13.28|16.807291934800322|
|(6,[0,1],[3.27,2.0])|     17.78| 18.62792502149313|
|(6,[0,1],[3.35,3.0])|     20.65| 22.19990859843958|
| (6,[0,1],[3.6,3.0])|     24.06|23.027469092390863|
| (6,[0,1],[9.0,4.0])|     48.33| 44.20993998062048|
|[1.17,2.0,0.0,1.0...|     32.83|14.099172366560746|
|[1.36,3.0,1.0,0.0...|     18.64|13.468703095805566|
|[1.44,2.0,0.0,0.0...|      7.56|11.192665693096684|
|[1.5,2.0,0.0,1.0,...|     11.59|15.191552218576431|
|[1.5,2.0,1.0,0.0,...|     10.65|10.624972753536236|
|[1.5,2.0,1.0,0.0,...|     11.17|10.624972753536236|
|[1.57,2.0,0.0,0.0...|     15.42|12.986714067559692|
|[1.67,3.0,1.0,0.0...|     10.33|15.8585950259

In [0]:
regressor.coefficients

Out[19]: DenseVector([3.3102, 3.3072, -0.7663, 2.4228, -0.0138, -1.3499])

In [0]:
regressor.intercept

Out[20]: 1.1891053228463426

In [0]:
pred_result.meanAbsoluteError,pred_result.meanSquaredError

Out[23]: (4.383311504621776, 35.78563910376075)